## Tasking Workflow

In this example workflow, a user will be shown how to:
* View pipelines
* Create a tasking request
* View a tasking request
* Cancel a tasking request

#### Initialize the Maxar Geospatial Platform SDK Module

In [ ]:
from MGP_SDK.interface import Interface

try:
  interface = Interface() # if .MGP-config was created
except:
  interface = Interface('<user_name>','<user_password>', '<user_client_id>') # if .MGP-config was not created

#### Initialize the Tasking portion of the Maxar Geospatial Platform SDK

In [ ]:
mgp_tasking = interface.tasking_service

#### Getting a Pipeline

Before we begin, we will want to find a pipeline to work with and the necessary information needed for the pipeline. If you are unfamiliar with a pipeline and what is needed for the order to work, you can use the get_pipeline_details function to determine what information needs to be passed in.

In [ ]:
mgp_ordering = interface.order_service

In this case, we want to use the *map-ready* pipeline with the namespace of *imagery*. With our pipeline identified, we can narrow down the response of the get_pipeline_details function to show us the necessary information needed to place a tasking request

In [ ]:
map_pipeline = mgp_ordering.get_pipeline_details(namespace='imagery', name='map-ready')
required_info = map_pipeline['data']['settings_schema']['required']
print("The required information needed to place an order on this pipeline is: {}".format(required_info))
for info in required_info:
    type_info = map_pipeline['data']['settings_schema']['properties'][info]
    print("The type of data for {} is: {}".format(info, type_info))

#### Create a Tasking Request

Before we create a tasking request, we will need to import the datetime module to make it easier to get UTC dates in ISO-8601 format

In [ ]:
import datetime

now_start = datetime.datetime.utcnow() + datetime.timedelta(minutes=10)
now_time = now_start.isoformat()
end_start = datetime.datetime.utcnow() + datetime.timedelta(days=1)
end_time = end_start.isoformat()
now = now_time.replace(now_time[-7:], "+00:00")
end = end_time.replace(now_time[-7:], "+00:00")
print(now)
print(end)

Having set our beginning and ending times, we now need to set up our tasking template. Using the information gathered from the pipeline request, we can construct a template. *NOTE: The inventory_ids field needs to be included in the tasking request, but no ids need to be passed in. Instead, use the string shown in the list as a placeholder*

In [ ]:
settings = {"inventory_ids": ["$.id"], "customer_description": "Your description"}
output_config = {"amazon_s3": {"bucket": "yourS3BucketName", "prefix": "your/S3/bucket/prefix/name"}}
notifications = {"type": "email", "target": "your.email@address.com", "level": "INITIAL_FINAL"}

In [ ]:
template = [{
    "pipeline": "imagery/map-ready",
    "template": {
        "settings": settings,
        "output_config": output_config,
        "notifications": [notifications],
        "metadata": {
            "project_id": "Your tasking name"
        }
    }
}]

We will also set our geoJSON in advance to use in the tasking request

In [ ]:
coordinates = {
    "type": "Polygon", 
    "coordinates": [
        [
            [152.931404, -27.556982],
            [153.172073, -27.556982],
            [153.172073, -27.394936],
            [152.931404, -27.394936],
            [152.931404, -27.556982]
        ]
    ]
}

With our variables set, we can now submit a tasking request. First, let's run a validation on our tasking request to make sure that everything is formatted correctly. We will be using the 50cm_Color recipe for this example

In [ ]:
tasking_validation = mgp_tasking.new_tasking(start_datetime=now, end_datetime=end, aoi_geojson=coordinates, 
                                             recipe='50cm_Color', order_templates=template, validate=True)
print(tasking_validation)

With our validaton passing, we can now run the full tasking request

In [ ]:
tasking = mgp_tasking.new_tasking(start_datetime=now, end_datetime=end, aoi_geojson=coordinates, 
                                  recipe='50cm_Color', order_templates=template)
tasking_id = tasking['data']['id']
print(tasking)

#### Viewing Tasking Requests

The tasking request has been submitted and the tasking_id has been set to a variable for use further along in the workflow. With the request submitted, we can now check on the status of the request with the get_tasking_request function

In [ ]:
my_tasking = mgp_tasking.get_tasking_request(tasking_id=tasking_id)
print(my_tasking)

#### Cancel a Tasking Request

In some cases, we may want to cancel a tasking order before it triggers. We can acomplish this with the cancel_tasking funciton

In [ ]:
cancel_tasking = mgp_tasking.cancel_tasking(tasking_id=tasking_id, reason="Your reason for cancelling")
print(cancel_tasking)